In [92]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

import openpyxl as op

In [93]:
# Paramétrage :

# from final_tab.ipynb
# Define the angle range and step
start_angle = -60
end_angle = 60
step_angle = 20

# Create the array of angles
angles = np.arange(start_angle, end_angle + 1, step_angle)
print(angles)

# Paramètres annuels
###### # from final_tab.ipynb
start_date = datetime(year = 2023, month = 1, day = 1, hour = 0)
end_date = datetime(year = 2024, month = 1, day = 1, hour = 0)
print((end_date-start_date).days)

# Paramètres horaires
###### # from final_tab.ipynb
HEURE_DEB = 6
HEURE_FIN = 22
HEURE_BLANCHES = 24 - (HEURE_FIN-HEURE_DEB)
NB_HEURES_ANNEES_ETUD = (HEURE_FIN-HEURE_DEB)*(end_date-start_date).days
TOT_HOYS = (end_date-start_date).days*24

[-60 -40 -20   0  20  40  60]
365


In [94]:
# Lecture du tableau généré aux étapes précédentes
df = pd.read_excel("results/tabmean_5840_step20.xlsx",sheet_name="Sheet1",header=None)

df.columns = angles

In [95]:
# Étape 1 : Ajouter autant de lignes nulles qu'HEURE_DEB si positif
if HEURE_DEB > 0:
    null_rows_start = pd.DataFrame(0, index=range(HEURE_DEB), columns=angles)
    df2 = pd.concat([null_rows_start, df], ignore_index=True)

# Étape 2 : Ajouter 8 lignes vides toutes les 16 lignes jusqu'à atteindre 8760 lignes
if HEURE_BLANCHES > 0:
    empty_rows = pd.DataFrame(0, index=range(HEURE_BLANCHES), columns=angles)

    for idx in range(HEURE_FIN, TOT_HOYS, 24):
        df2 = pd.concat([df2.iloc[:idx], empty_rows, df2.iloc[idx:]], ignore_index=True)

# Étape 3 : Enlever les lignes en trop
if df2.shape[0] > TOT_HOYS :
    surplus = df2.shape[0] - TOT_HOYS
    df3 = df2.iloc[:-surplus]

In [96]:
print(df3[0:24])

         -60       -40       -20        0         20        40        60
0   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
1   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
2   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
3   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
4   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
5   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
6   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
7   0.049168  0.049164  0.049165  0.049168  0.049168  0.049161  0.049168
8   0.181327  0.181314  0.181316  0.181327  0.181327  0.181300  0.181327
9   0.312087  0.312063  0.312066  0.312087  0.312087  0.312036  0.312087
10  0.343043  0.343010  0.343014  0.343043  0.343043  0.342973  0.343043
11  0.305425  0.305392  0.305396  0.305425  0.305425  0.305354  0.305425
12  0.320081  0.320049  0.320053  0.320081  0.32008

In [97]:
# Créer un DataFrame avec les heures de l'année (de 0 à 8759) en tant qu'index
hours_in_year = range(8760)
dfX = pd.DataFrame(index=hours_in_year)

# Ajouter la colonne 'month_day_hour' sans déclencher l'avertissement
df3['month_day_hour'] = pd.to_datetime(dfX.index.map(lambda x: start_date + timedelta(hours=x)))

# Réorganiser les colonnes dans df3
cols = ['month_day_hour'] + [col for col in df3 if col != 'month_day_hour']
df3 = df3[cols]
print(df3.head())

       month_day_hour  -60  -40  -20    0   20   40   60
0 2023-01-01 00:00:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1 2023-01-01 01:00:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2 2023-01-01 02:00:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3 2023-01-01 03:00:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0
4 2023-01-01 04:00:00  0.0  0.0  0.0  0.0  0.0  0.0  0.0


c:\Users\maceo.valente\.virtualenvs\TSAgriPV-XqNYGekU\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Import scenario :
1 = True = Backtracking (productible max)
0 = False = Effacement (irradiance au sol maximum)

In [98]:
from scenario import Scenario

## TESTS ##
# Instancier des objets scénarios
s1 = None
xl_path = "scenarios/sc_test1.xlsx"
s1 = Scenario(file_path=xl_path)
# print(s1.scenario[:,:])

s2 = Scenario(file_path=xl_path)
print(s2.scenario[:,:])



[[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True False
  False False False False False False  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True False
  False False False False False False  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True False
  False  True  True  True False 

In [99]:
s2.scenario.shape

(12, 24)

Création équivalent 1/0

In [100]:
# Création_tab 1
sc10 = np.full((12,24), 0, dtype = "int8")

for i in range(s2.scenario.shape[0]):  # Pour chaque ligne
    for j in range(s2.scenario.shape[1]):  # Pour chaque colonne
        if s2.scenario[i, j] == True:
            sc10[i, j] = 1

print(sc10)

[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 0 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]


Création tab_1

In [101]:
# Nombre de lignes souhaité
n_rows = 8760

# Création d'un DataFrame avec des valeurs par défaut (par exemple, 0)
tab_1 = pd.DataFrame({
    'hoys': df3['month_day_hour'],
    'effacement': np.ones(n_rows),
    'angle': np.zeros(n_rows),
    'irradiance_sol': np.zeros(n_rows)
})


Remplir le tab_1 :

In [102]:
# Remplissage de la colonne 'effacement'
# Parcourir scénario, pour chaque 0 (False), changer les 1 (panneaux en suntracking) de la colonne 'effacement' en 0 (panneaux effacés)

for i in range(s1.scenario.shape[0]) :  # Pour chaque ligne
    for j in range(s1.scenario.shape[1]) :  # Pour chaque colonne
        if s1.scenario[i,j] == False :
            tab_1.loc[(tab_1['hoys'].dt.hour == j) & (tab_1['hoys'].dt.month == i+1), 'effacement'] = 0

In [104]:
# Remplissage de la colonne 'angle'
# Parcourir tableau_1, en fonction de 'effacement', remplir 'angle' et 'irradiance'
for i in range(tab_1.shape[0]) :
    if tab_1.at[i,'effacement'] == 0:
        #aller chercher irradiance au sol maximum dans tab_final (df3 dans ce .ipynb), et retrouver l'angle correspondant (nom de colonne)
        tab_1.loc[i,'irradiance_sol'] = df3.drop(columns='month_day_hour').iloc[i].max() # Irradiance max
        tab_1.loc[i,'angles'] = df3.drop(columns='month_day_hour').iloc[i].idxmax() # Colonne dont l'irradiance est max
        # /!\ Si égalité entre plusieurs angles, va choisir l'angle le moins élevé (-60 à priori)
    #else: #'effacement == 1
        # Aller chercher angle PVSYST optimal, arrondir, puis trouver irradiance au sol correspondante
  

In [105]:
# print(tab_1[(tab_1['effacement'] < 1) & (tab_1['hoys'].dt.month == 9)])
print(tab_1[5840:5870])

                    hoys  effacement  angle  irradiance_sol  angles
5840 2023-09-01 08:00:00         1.0    0.0        0.000000     NaN
5841 2023-09-01 09:00:00         1.0    0.0        0.000000     NaN
5842 2023-09-01 10:00:00         1.0    0.0        0.000000     NaN
5843 2023-09-01 11:00:00         0.0    0.0        0.379713   -60.0
5844 2023-09-01 12:00:00         1.0    0.0        0.000000     NaN
5845 2023-09-01 13:00:00         1.0    0.0        0.000000     NaN
5846 2023-09-01 14:00:00         1.0    0.0        0.000000     NaN
5847 2023-09-01 15:00:00         1.0    0.0        0.000000     NaN
5848 2023-09-01 16:00:00         1.0    0.0        0.000000     NaN
5849 2023-09-01 17:00:00         0.0    0.0        0.213432   -60.0
5850 2023-09-01 18:00:00         1.0    0.0        0.000000     NaN
5851 2023-09-01 19:00:00         1.0    0.0        0.000000     NaN
5852 2023-09-01 20:00:00         1.0    0.0        0.000000     NaN
5853 2023-09-01 21:00:00         1.0    0.0     

In [108]:
print(df3.iloc[5867,:])

month_day_hour    2023-09-02 11:00:00
-60                          0.434549
-40                          0.434496
-20                          0.434502
0                            0.434549
20                           0.434549
40                           0.434436
60                           0.434549
Name: 5867, dtype: object
